In [1]:
import tensorflow as tf
import core.preprocessing as pp
from core.net_models import FC, CNN

LOGDIR = "./tmp/debug/"

present = 5*12
future = 3*12

dm = pp.DataManager()
dm.load_all_and_split(kind='small')
dftrain = dm.detrended_train
dfdev = dm.detrended_dev

# Drop columns zeroed out by detrend transformation
for item in pp.DROP_LIST:
    key = 'S{:02}'.format(item)
    del dftrain[key]
    del dfdev[key]

nseries = dftrain.shape[1] - 1

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
def submit(net_model, niter, lr, fname):
    # Start clean
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders for input and output
    x = tf.placeholder(tf.float32, shape=[None, present*nseries], name="x")
    y = tf.placeholder(tf.float32, shape=[None, future], name="y")
    
    # Similarly, setup placeholders for dev set
    xdev = tf.placeholder(tf.float32, shape=[None, present*nseries], name="xdev")
    ydev = tf.placeholder(tf.float32, shape=[None, future], name="ydev")
    
    # Setup placeholders for day of year
    day = tf.placeholder(tf.float32, shape=[None, 1], name="day")
    daydev = tf.placeholder(tf.float32, shape=[None, 1], name="daydev")
    
    # Feed forward training and dev examples
    if net_model is 'FC':
        nn = FC([1024,512,future])
    elif net_model is 'CNN':
        nn = CNN([32,64,future], height=nseries, width=present)

    yhat    = nn(x)
    yhatdev = nn(xdev)

    # Define loss in training and dev set
    with tf.name_scope("loss"):
        train_loss = tf.losses.mean_squared_error(labels=y, predictions=yhat)
        dev_loss = tf.losses.mean_squared_error(labels=ydev, predictions=yhatdev)
        tf.summary.scalar("train_loss", train_loss)
        tf.summary.scalar("dev_loss", dev_loss)

    # Minimize training loss
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(lr).minimize(train_loss)

    # Collect all summaries for TensorBoard
    summ = tf.summary.merge_all()

    # Start of execution
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + fname)
    writer.add_graph(sess.graph)

    for i in range(niter):
        # create batch
        X, Y, D = pp.make_batch(dftrain, 100, present, future)
        Xdev, Ydev, Ddev = pp.make_batch(dfdev, 100, present, future)
        
        # center data
        X = pp.center_design_matrix(X)
        Xdev = pp.center_design_matrix(Xdev)
        
        if i % 5 == 0:
            [tloss, dloss, s] = sess.run([train_loss, dev_loss, summ],
                                         feed_dict={x: X, y: Y, day: D,
                                                    xdev: Xdev, ydev: Ydev, daydev: Ddev})
            writer.add_summary(s, i)
            writer.flush()
        
        sess.run(train_step, feed_dict={x: X, y: Y, day: D,
                                        xdev: Xdev, ydev: Ydev, daydev: Ddev})

In [4]:
net_models = ['CNN', 'FC']

def main():
    for lr in [1e-2, 1e-3, 1e-4]:
        for net_model in net_models:
            niter = 100
            fname = "{},lr={}".format(net_model, lr)
        
            submit(net_model, niter, lr, fname)
    
main()